In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [2]:
def load_and_preprocess(filepath, is_test=False):
    data = pd.read_csv(filepath, delimiter=' ::: ', engine='python', header=None)
    if is_test:
        data.columns = ['ID', 'TITLE', 'DESCRIPTION']
    else:
        data.columns = ['ID', 'TITLE', 'GENRE', 'DESCRIPTION']
    data['DESCRIPTION'] = data['DESCRIPTION'].str.lower().str.replace('[^\w\s]', '', regex=True)
    return data

In [3]:
train_data = load_and_preprocess('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt')
test_data = load_and_preprocess('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt', is_test=True)
test_solution_data = load_and_preprocess('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt')

In [4]:
X_train = train_data['DESCRIPTION']
y_train = train_data['GENRE']
X_test = test_data['DESCRIPTION']
y_test = test_solution_data['GENRE']

In [5]:
X_train

0        listening in to a conversation between his doc...
1        a brother and sister with a past incestuous re...
2        as the bus empties the students for their fiel...
3        to help their unemployed father make ends meet...
4        the films title refers not only to the unrecov...
                               ...                        
54209    this shortlived nbc live sitcom centered on bo...
54210    the next generation of exploitation the sister...
54211    ze bestaan echt is a standup comedy about grow...
54212    walter and vivian live in the country and have...
54213    on labor day weekend 1935 the most intense hur...
Name: DESCRIPTION, Length: 54214, dtype: object

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [7]:
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

MultinomialNB()

In [8]:
from sklearn.metrics import accuracy_score
nb_predictions = nb_model.predict(X_test_tfidf)

In [9]:
print(f"Accuracy: {accuracy_score(y_test, nb_predictions)*100:.4f}%")

Accuracy: 44.3948%
